# Coroutines: Into the Async

<br>

<img src="./images/into_the_unknown.jpeg" width="80%">

# Contents<a id="contents"></a>

* [From Generators to Coroutines](#coroutines)
  * [From One Generator to Another: "yield from"](#yield-from)
  * [Two-Faced "yield", or Generator's "next" vs. Coroutine's "send"](#next-vs-send)
  * [GeneratorExit, or The Moment When the Coroutine/Generator Stops](#debt1)
  * [Several Coroutines: Scheduler (Orchestrator) Example](#scheduler)
  * [Two in One: Receive & Send (But Try Not to Do That)](#debt2)
  * [Don't Throw an Error at Me – Better Send Me a Signal](#signal)

## From Generators to Coroutines<a id="coroutines"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

<img src="./images/beazley_intro_slide.png" width="80%">

Приставка "co" в слове корутина — означает "совместно", "вместе".
Ключевую роль играет возможность передачи значения другим корутинам (`send`) и ожидание результата работы от других корутин (`yield from`).

Из [словаря](https://en.wiktionary.org/wiki/coroutine):
> *Coroutine* is a piece of code that performs a task, and that can be passed new input and return output more than once.

И в качестве более частных разновидностей корутины можно выделить *сабрутины* (обычные программы):
> Subroutine is a coroutine that accepts input once and returns output once.

и *генераторы*:
> Generator is a coroutine that accepts input once, but yields output multiple times.

Таким образом, корутина как бы использует часть функционала от генератора, расширяя его.

А вообще, на корутины и обычные функции можно смотреть просто как на абстракции стека вызовов и порядка выполнения инструкций: когда есть некоторый порядок выполнения блоков кода и в каждом блоке известна текущая выполняемая инструкция.

<img src="./images/call_stack.png" width="80%">

*Пример-иллюстрация концепции Call stack-а на JavaScript (https://ru.hexlet.io/courses/js-asynchronous-programming/lessons/call-stack/theory_unit).*

Как и в случае сабрутин, при выполнении корутин выделяется память под локальные переменные, последовательно выполняются строки.
На стадии `return` обычной функции возвращаемое значение кладётся в стек вызовов, и освобождается память, выделенная под локальные переменные, под указатель на текущую инструкцию в теле функции.
Отличие же корутин от обычных функций в том, что при выбрысывании значения вовне с помощью `yield` оно кладётся в стек, но *состояние корутины сохраняется*.
При получении значения из `yield` состояние стека восстанавливается, и продолжается выполнение корутины.
Таким образом, корутина может передавать контроль другой функции с сохранением собственного состояния.

Сходство программы, где вызываются корутины, с программой, где работают простыми функциями, в том, что порядок выполнения кода в обоих случаях последовательный (выполняется одна функция, после неё выполняется другая, но не случайная, а конкретная).
Таким образом, корутины, запущенные из одного источника, могут одновременно находиться в состоянии выполнения, но в каждый момент времени реально выполняется только одна из них ("concurrent but not parallel").

Корутина не может передавать управление произвольной корутине, в то же время находящейся в состоянии исполнения.
Корутина может передать управление либо вновь созданной корутине (`yield from`), либо коду, вызвавшему корутину (который в стеке вызовов находится сразу перед корутиной).
И родитель корутины (например, сам *планировщик задач*) может уже передать контроль другой дочерней корутине.

### From One Generator to Another: `yield from`<a id="yield-from"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

In [1]:
def filter_greater_than_zero(values):
    for v in values:
        if v >= 0:
            yield v

def filter_lower_than_zero(values):
    for v in values:
        if v < 0:
            yield v

def complex_generate(values):
    yield from filter_greater_than_zero(values)
    yield from filter_lower_than_zero(values)

In [2]:
for v in complex_generate(range(-3, 3)):
    print(v)

0
1
2
-3
-2
-1


### Two-Faced `yield`, or Generator's `next` vs. Coroutine's `send`<a id="next-vs-send"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

Функция `next` *получает* значение из генератора.

Метод же `send` *посылает* значение в корутину.

При этом `yield`, который внутри корутины, всегда *и посылает, и получает* значение.

Здесь важен порядок отправки-получения: в выражении с `yield` значение сначала выбрасывается из корутины вовне, и только потом значение выражения с `yield` становится равным тому значению, которое передано в корутину с помощью `send`.
```python
received_value = yield emitted_value
```

Метод `send` также двоякий: он не только отправляет одно значение, но и принимает другое из корутины.
```python
# coroutine = ...

received_value = coroutine.send(emitted_value)
```

Отсюда становится понятно, почему **при инициализации корутины приходится сначала делать `next()` или `send(None)`**: после этого происходит первый выброс значения из корутины и выполнение корутины останавливается перед получением значения в `yield`.
Далее, при следующем вызове `send` с уже нормальным значением выполнение корутины продолжается на этапе *получения значения из yield*, продолжается до следующего `yield`, из него выбрасывается значение, и код останавливается, опять, на моменте получения значения в новый `yield`.

С генераторами всё похоже, только там не надо делать "инициализацию", потому что первый выброс значения уже, как правило, осмысленный.

In [3]:
def receive_value():
    while True:
        received_value = yield

        print(f'Received: "{received_value}".')

In [4]:
coroutine = receive_value()

next(coroutine)

coroutine.send("Hello world!")

Received: "Hello world!".


### GeneratorExit, or The Moment When the Coroutine/Generator Stops<a id="debt1"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

Значения посылаются в корутины с помощью метода `send`.
Закрыть корутину от получения значений можно с помощью метода `close`.

In [5]:
coroutine.close()

После закрытия корутину использовать уже не получится:

In [6]:
try:
    coroutine.send("Hello world!")
except StopIteration:
    print('Failed to send value! (StopIteration).')

Failed to send value! (StopIteration).


То же самое можно было сделать и с генераторами:

In [7]:
def generate():
    while True:
        yield "Hello world!"

In [8]:
generator = generate()

print(next(generator))

generator.close()

try:
    print(next(generator))
except StopIteration:
    print('Failed to receive value! (StopIteration).')

Hello world!
Failed to receive value! (StopIteration).


На самом деле при `close` внутри корутины (генератора) выбрасывается ошибка `GeneratorExit`:

In [9]:
def receive_value():
    while True:
        try:
            received_value = yield
        except GeneratorExit as exception:
            print('Coroutine closed!')
            raise exception
        else:
            print(f'Received: "{received_value}".')

In [10]:
coroutine = receive_value()

next(coroutine)

coroutine.send("Hello world!")
coroutine.close()

Received: "Hello world!".
Coroutine closed!


Стоит отметить, что ошибка `GeneratorExit` наследуется не от `Exception`, а от `BaseException`, так как "[по сути не является ошибкой](https://docs.python.org/3/library/exceptions.html#GeneratorExit)".

### Several Coroutines: Scheduler (Orchestrator) Example<a id="scheduler"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

Сымитируем осмысленный пример использования корутин: определим несколько корутин, которые будут параллельно что-то делать и которые в нужный момент будет дёргать планировщик (оркестратор).

In [11]:
def sqr():
    while True:
        value = yield
        
        print(f'Output (sqr): {value ** 2}.')


def add2():
    while True:
        value = yield
        
        print(f'Output (add2): {value + 2}.')


# Creation
coroutine1 = sqr()
coroutine2 = add2()


# Initialization
next(coroutine1)
next(coroutine2)


# Scheduler — as manually written `for`-cycle
for v in range(-3, 3):
    # Several coroutines working: in parallel, but not simultaneously
    
    print(f'Input: {v}.')
    
    if v % 2 == 0:
        coroutine1.send(v)
    else:
        coroutine2.send(v)


# Finish
coroutine1.close()
coroutine2.close()

Input: -3.
Output (add2): -1.
Input: -2.
Output (sqr): 4.
Input: -1.
Output (add2): 1.
Input: 0.
Output (sqr): 0.
Input: 1.
Output (add2): 3.
Input: 2.
Output (sqr): 4.


### Two in One: Receive & Send (But Try Not to Do That)<a id="debt2"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

Можно написать корутину, которая и принимает и отдаёт значения.
Правда, лучше этим не увлекаться, потому что не понятно и легко ошибиться 😅

In [12]:
def receive_and_send():
    value = 'initial value'
    
    while True:
        received_value = yield value
        
        print(f'Sent: {value}. Received: {received_value}')
        
        value = received_value

In [13]:
coroutine = receive_and_send()

next(coroutine)

'initial value'

In [14]:
coroutine.send("Hello world!")

Sent: initial value. Received: Hello world!


'Hello world!'

In [15]:
coroutine.send("""
Tired of drifting, searching, shifting through town to town
Every time I slip and slide a little further down
I can't blame you if you won't take me back
After everything I put you through
But honey you're my last hope
And who else can I turn to
""")

Sent: Hello world!. Received: 
Tired of drifting, searching, shifting through town to town
Every time I slip and slide a little further down
I can't blame you if you won't take me back
After everything I put you through
But honey you're my last hope
And who else can I turn to



"\nTired of drifting, searching, shifting through town to town\nEvery time I slip and slide a little further down\nI can't blame you if you won't take me back\nAfter everything I put you through\nBut honey you're my last hope\nAnd who else can I turn to\n"

### Don't Throw an Error at Me – Better Send Me a Signal<a id="signal"></a>
<div style="text-align: right;"><a href=#contents>Back to Contents</a></div>

Ещё один инструмент управления корутинами (кроме методов `send` и `close`) — метод `throw`: "посылка сигнала".
Дело в том, что корутина — как небольшая отдельная программа.
И в случае, если "планировщик" решает, что этой программе надо сделать "что-то ещё", помимо её основной работы (`send`) или завершения (`close`), он отправляет соответствующий сигнал (сигнал, о котором должна быть в курсе сама корутина).

Сигналы реализуются в виде "исключений".
Но в этом не стоит усматривать что-то "противоестественное" ("костылеватое").
Ведь одна из основных (если не самая главная) задач вообще всех исключений — "что-то кому-то сообщить" (так, и ошибку часто бросают именно для того, чтобы её кто-то мог *перехватить и обработать*). 

In [16]:
class PrintHelloSignal(Exception):
    pass

class PrintWorldSignal(Exception):
    pass

In [17]:
def receive():
    while True:
        try:
            received_value = yield

            print(f'Received: {received_value}.')

        except PrintHelloSignal:
            print(f'Hello', end=' ')

        except PrintWorldSignal:
            print(f'world!')

In [18]:
coroutine = receive()

next(coroutine)

In [19]:
coroutine.send(-17.5)

Received: -17.5.


In [20]:
coroutine.throw(PrintHelloSignal)
coroutine.throw(PrintWorldSignal)

Hello world!


In [21]:
coroutine.send("Bye")

Received: Bye.


---

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## References

* [Silver Lady](https://genius.com/David-soul-silver-lady-lyrics) by David Soul.